<a href="https://colab.research.google.com/github/praneetha-ch/StressDetectionUsingFacialEmotionRecognition/blob/main/StressDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Modules

In [ ]:
import gdown
from google.colab import drive
drive.mount('/content/drive',force_remount=True, timeout_ms=360000)

!cp '/content/drive/MyDrive/CK+.zip' "CK+48"
!unzip "/content/CK+48" -d /content/CK

In [ ]:
import os
import numpy as np
import matplotlib.image as mpimg

import random
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array


#Loading the Dataset and Counting Images

In [ ]:

# Define the main path to the dataset
dataset_path = '/content/CK/ck/CK+48/'


In [ ]:
import cv2
from sklearn.model_selection import train_test_split

stress_emotions = ["anger", "contempt", "disgust", "fear", "sadness"]
non_stress_emotions = ["happy", "surprise"]

X = []
y = []

for folder in os.listdir(dataset_path):
  images_path = os.path.join(dataset_path, folder)
  images = os.listdir(images_path)
  for image in images:
    img_arr = cv2.imread(images_path+"/"+image)
    X.append(img_arr)
    if folder in stress_emotions:
      y.append(1)
    else:
      y.append(0)
X = np.array(X, dtype=int)
y = np.array(y, dtype=int)

y = tf.keras.utils.to_categorical(y, num_classes=2)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)


#ResNet50 Model

In [ ]:

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top of the ResNet50 base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.build(input_shape=(None, 48, 48, 3))
print(model.summary())

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)



In [ ]:
import datetime

# Record the start time before training begins
start_time = datetime.datetime.now()

# Your model training code here
# model.fit(X_train, y_train)



# Calculate the total time taken for training

# Train the model
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=100,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpoint, reduce_lr, early_stopping])
end_time = datetime.datetime.now()

time_taken = end_time - start_time
print(f"Time taken for training: {time_taken}")

# Save the model weights
model.save_weights('/content/best_model.weights.h5')

In [ ]:
import seaborn as sns
from sklearn.metrics import accuracy_score

def plot_accuracies(history):
    plt.plot(history.history['accuracy'], label="train")
    plt.plot(history.history['val_accuracy'], label="val")
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    #plt.xticks(history.epoch, rotation=90)
    plt.xlabel("Epochs")
    plt.legend(loc='upper left')
    plt.show()

def plot_losses(history):
  plt.plot(history.history['loss'], label="train")
  plt.plot(history.history['val_loss'], label="val")
  plt.title('Model Loss')
  plt.ylabel('Loss')
  #plt.xticks(history.epoch, rotation=90)
  plt.xlabel("Epochs")
  plt.legend(loc='upper right')
  plt.show()

def plot_confusion_matrix(model, X_test, y_test):
  y_pred = model.predict(X_test)
  y_pred = np.argmax(y_pred, axis=1)
  y_test = np.argmax(y_test, axis=1)
  confusion_mat = confusion_matrix(y_test, y_pred)
  sns.heatmap(confusion_mat, annot=confusion_mat, xticklabels=["non stress", "stress"],
              yticklabels=["non stress", "stress"], fmt=".0f")
  plt.show()

In [ ]:
plot_accuracies(history)
plot_losses(history)
plot_confusion_matrix(model, X_test, y_test)

#VGG16 Model

In [ ]:

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top of the VGG16 base model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.build(input_shape=(None, 48, 48, 3))
print(model.summary())

checkpoint = ModelCheckpoint("best_model_VGG.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

In [ ]:
# Train the model
import datetime
start_time=datetime.datetime.now()
history_vgg = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=50,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpoint, reduce_lr, early_stopping])
end_time=datetime.datetime.now()
time_taken=end_time-start_time
print(f"Time taken for training: {time_taken}")

# Save the model weights
model.save_weights('/content/best_model_vgg.weights.h5')

In [ ]:
plot_accuracies(history_vgg)
plot_losses(history_vgg)
plot_confusion_matrix(model, X_test, y_test)

#Proposed Model

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 3rd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.22))

model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())

checkpoint = ModelCheckpoint("best_model_new_arch.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True)

In [ ]:
import datetime

# Record the start time before training begins
start_time = datetime.datetime.now()

# Your model training code here
# model.fit(X_train, y_train)


history_new_arch = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=100,
                    validation_data=(X_test, y_test),
                    callbacks=[checkpoint, reduce_lr, early_stopping])
# Record the end time after training is done
end_time = datetime.datetime.now()

# Calculate the total time taken for training
time_taken = end_time - start_time
print(f"Time taken for training: {time_taken}")

In [ ]:
plot_accuracies(history_new_arch)
plot_losses(history_new_arch)
plot_confusion_matrix(model, X_test, y_test)